# Solving continuous time economic models: Diffusion income process

This Jupyter notebook presents Julia code that solves a continuous time version of a heterogeneous agents economy. Specifically, it solves a version of the Huggett model in which agents solve a consumption-savings problem subject to a borrowing constraint and idiosyncratic shocks. This version of the model assumes that the income process for agents follows a Diffusion process across income states.

The model and its solution method is described in detail in the accompanying PDF document. Note, too, that an additional Jupyter notebook solves the same model but with a Poisson process governing the income state variable. 

#### Defining a composite type for the model object

The first thing we do before anything else is declare a composite type, which contains (most of) the fields that will be used in the solution to the model. Each model parameter, variable, and data storage matrix is given a particular type, which helps increase the speed with which the code can be run. 

In [1]:
type Huggett_model
  gamma::Float64            # risk aversion coefficient
  rho::Float64              # discount rate.
  ou_var::Float64           # Ornstein Uhlenbeck variance
  zmean::Float64            # Mean of log-normal distriution for the Ornstein Uhlenbeck process
  ou_corr::Float64          # Ornstein Uhlenbeck correlation
  theta::Float64            # Ornstein Uhlenbeck param
  sig2::Float64             # Ornstein Uhlenbeck variance
  I::Int64                  # Number of grids for assets
  J::Int64                  # Number of grids for income
  Ir::Int64                 # Number of interest rate iterations
  amin::Float64             # Min assets = borrowing constraint
  amax::Float64             # Max assets
  zmin::Float64             # Min income
  zmax::Float64             # Max income
  rmin::Float64             # Min interest rate
  rmax::Float64             # Max interest rate
  agrid::Array{Float64}     # Grid space for assets
  zgrid::Array{Float64}     # Grid space for incoomes
  aa::Array{Float64}        # Matrix for asset grid space
  zz::Array{Float64}        # Matrix for income grid space
  da::Float64               # Asset grid step size
  dz::Float64               # Income grid step size
  dz2::Float64              # Income grid step size squared
  Delta::Float64            # Time step-size
  mu::Array{Float64}        # Drift from Ito's lemma
  s2::Array{Float64}        # Std Dev from Ito's lemma
  chi::Array{Float64}       # elements in the transition matrix
  nu::Array{Float64}        # elements in the transition matrix
  zeta::Array{Float64}      # elements in the transition matrix
  Cswitch::SparseMatrixCSC{Float64,Int64}  # Transition matrix
  v0::Array{Float64}        # Initial guess for value function
  r0::Float64               # Intial guess for interest rate
  maxit::Int64              # Max iterations for solving HJB
  crit::Float64             # Tolerance for HJB equation solution  
  crit_S::Float64           # Tolerance for asset mkt clearing
end


Next we create the model object itself. We do this by defining a function, which takes the deep parameters of the model as default arguments, and produces all of the other model objects that can be pre-computed. This includes the grid spaces for assets and income, the parameters of the Ornstein-Uhlenbeck difussion process for income, the initial guess for the HJB value function, and the transition matrix used in computation of the HJB equation.

The final command in this cell produces an instance of the model object (`Huggett_model`), which we will use when calling any of the paramters, variables, data storage matrices, or pre-computed model objects. 

In [2]:
function Huggett_model(;gamma::Float64=2.0,
  rho::Float64=0.05,
  ou_var::Float64=0.05,
  ou_corr::Float64=0.95,
  I::Int64=100,
  J::Int64= 100,
  Ir::Int64=40,
  amin::Float64=-0.10,
  amax::Float64=5.0,
  rmin::Float64=0.001,
  rmax::Float64=0.045,
  maxit::Int64=100,
  crit::Float64=1e-6,
  crit_S::Float64=1e-5,
  Delta::Float64=1000.0,
  r0::Float64=0.03)

  # Ornstein-Uhlenbeck process: dlog(z) = -theta*log(z)dt + sig*dW
  zmean   = exp(ou_var)
  theta   = -log(ou_corr)
  sig2    = theta*ou_var

  zmin    = zmean*0.2
  zmax    = zmean*1.5
  agrid   = reshape(collect(linspace(amin,amax,I)),I,1)
  zgrid   = reshape(collect(linspace(zmin,zmax,J)),J,1)
  aa      = repmat(agrid,1,J)
  zz      = repmat(zgrid',I,1)
  da      = (amax-amin)/(I-1)
  dz      = (zmax-zmin)/(J-1)
  dz2     = dz^2
  # Ornstein-Uhlenbeck: in logs
  mu      = (-theta*log(zgrid) + sig2/2).*zgrid
  s2      = sig2.*zgrid.^2
  
  chi     = s2/(2*dz2)
  nu      = - s2/dz2 - mu/dz
  zeta    = mu/dz + s2/(2*dz2)

  v0      = (zz + r0.*aa).^(1-gamma)/(1-gamma)/rho

  #This will be the upperdiagonal of the C_switch
  updiag    = zeros(I,1)
  for j in 1:J-1
      updiag=vcat(updiag, repmat([zeta[j]],I,1))
  end

  #This will be the center diagonal of the B_switch
  centdiag = repmat([chi[1]+nu[1]],I,1)
  for j in 2:J-1
      centdiag = vcat(centdiag, repmat([nu[j]],I,1))
  end
  centdiag = vcat(centdiag, repmat([nu[J]+zeta[J]],I,1))

  #This will be the lower diagonal of the B_switch
  lowdiag = repmat([chi[2]],I,1)
  for j in 3:J
      lowdiag = vcat(lowdiag, repmat([chi[j]],I,1))
  end

  # Add upper, center, and lower diagonal into a sparse matrix
  Cswitch     = spdiagm(centdiag[:,1],0,I*J,I*J) + spdiagm(lowdiag[:,1],-I,I*J,I*J) + spdiagm(updiag[I+1:end,1],I,I*J,I*J)

  Huggett_model(gamma, rho, ou_var, zmean, ou_corr, theta, sig2, I, J, Ir, amin, amax, zmin, zmax, rmin, rmax, agrid, zgrid, aa, zz, da, dz, dz2, Delta, mu, s2, chi, nu, zeta, Cswitch, v0, r0, maxit, crit, crit_S)
end

# Creates an instance of the model
mod = Huggett_model();

Next I define several useful functions for use in solving the model. The first three are, the CRRA utility function, the derivative of the utility function, and the inverse of the derivative of the utility function. The final function solves the Kolmogorov Forward Equation for the model using the transition matrix found by solving the HJB equation. 

In [3]:
function util(x; gamma::Float64=mod.gamma)
  u = x.^(1-gamma)/(1-gamma)
  return u
end

function utilprime(x; gamma::Float64=mod.gamma)
  uprime = x.^(-gamma)
  return uprime
end

function utilprimeinv(x; gamma::Float64=mod.gamma)
  uprimeinv = x.^(-1/gamma)
  return uprimeinv
end

function KFE(A::SparseMatrixCSC{Float64,Int64}; I::Int64=mod.I, J::Int64=mod.J, da::Float64=mod.da, dz::Float64=mod.dz)
  AT          = A'
  b           = zeros(I*J,1)
  i_fix       = 1    # fix one value so that matrix is not singular
  b[i_fix]    = 0.1
  row         = hcat(zeros(1,i_fix-1), 1, zeros(1,I*J-i_fix))
  AT[i_fix,:] = row
  #Solve linear system
  gg          = AT\b          # A'*gg = b --> gg = (A')^(-1)*b
  g_sum       = gg'*ones(I*J,1)*da*dz
  gg          = gg./g_sum     # Normalize so sum(gg)*da) = 1
  g           = reshape(gg,I,J)
  return g
end


KFE (generic function with 1 method)

Next, we initialize several data storage arrays. This turns out to be important, as Julia does not store arrays/data that are initialized and produced inside loops. Thus, anything that we might want access to after convergence of the algorithm (e.g. the equilibrium interest rate, the value function, policy functions) needs to be initialized here. 

In [5]:
Vaf         = zeros(mod.I,mod.J)
Vab         = zeros(mod.I,mod.J)
Vzf         = zeros(mod.I,mod.J)
Vzb         = zeros(mod.I,mod.J)
Vzz         = zeros(mod.I,mod.J)
c           = zeros(mod.I,mod.J)
r_r         = zeros(mod.Ir,1)
rmin_r      = zeros(mod.Ir,1)
rmax_r      = zeros(mod.Ir,1)
g_r         = zeros(mod.I,mod.J,mod.Ir)    # Stationary distribution
adot        = zeros(mod.I,mod.J,mod.Ir)    # Policy function
V_r         = zeros(mod.I,mod.J,mod.Ir)    # Value function
S           = zeros(mod.Ir,1)              # Aggregate asset supply
V           = zeros(mod.I,2)               # Value function
A           = sparse(zeros(mod.J*mod.I,mod.J*mod.I))   # Transition matrix
r           = mod.r0                       # Current guess for the interest rate
dist        = zeros(mod.maxit,1)           # Distance between guesses for the value function
r_end       = 0                            # Last iteration value

0

Now we get to the meat of the code. This code block runs the algorithm for solving the model and the associated equilibrium. 

#### Inner loop of the algorithm
After the outer loop has updated (initalized) the interest rate guess, the inner loop proceeds as follows:
* Construct the forward difference approximations of the derivatives of the value function
* Use the "upwind scheme" to compute consumption, utility, and the savings policy rules
* Use the savings policy rules to construct the transition matrix. This construction is carried out using sparse matrix methods to save memory (these methods are also fast). 
* Construct and solve the linear system of equations that constitute the HJB equation(s) across asset and income states.

#### Outer loop of the algorithm
After the inner loop has solved the HJB equation, the transition matrix computed on the final iteration is used to solve the KFE for the stationary distribution. The outer loop then uses this stationary distribution to compute the aggregate asset supply. We then update the interest rate via a bisection method until assets are in zero net supply. 

In [6]:
for ir in 1:mod.Ir
  r_r[ir]     = r
  rmin_r[ir]  = mod.rmin
  rmax_r[ir]  = mod.rmax

  if ir>1
    mod.v0 = V_r[:,:,ir-1]
  end

  v = mod.v0

  for n in 1:mod.maxit
    V               = v
    # forward difference
    Vaf[1:mod.I-1,:]    = (V[2:mod.I,:]-V[1:mod.I-1,:])/mod.da
    Vaf[mod.I,:]        = utilprime(mod.zgrid + r.*mod.amax)
    # backward difference
    Vab[1,:]            = utilprime(mod.zgrid + r.*mod.amin)  #state constraint boundary condition
    Vab[2:mod.I,:]      = (V[2:mod.I,:]-V[1:mod.I-1,:])/mod.da

    #consumption and savings with forward difference
    cf              = utilprimeinv(Vaf)
    sf             = mod.zz + r.*mod.aa - cf
    #consumption and savings with backward difference
    cb              = utilprimeinv(Vab)
    sb             = mod.zz + r.*mod.aa - cb
    #consumption and derivative of value function at steady state
    c0              = mod.zz + r.*mod.aa
    Va0             =utilprime(c0)

    # dV_upwind makes a choice of forward or backward differences based on the sign of the drift
    If              = sf .> 0    # positive drift --> forward difference
    Ib              = sb .< 0    # negative drift --> backward difference
    I0              = (1-If-Ib)   # at steady state
    Va_Upwind       = Vaf.*If + Vab.*Ib + Va0.*I0    #important to include third term
    c               = utilprimeinv(Va_Upwind)
    u               = util(c)

    #CONSTRUCT MATRIX
    X               = - min(sb,0)/mod.da
    Y               = - max(sf,0)/mod.da + min(sb,0)/mod.da
    Z               = max(sf,0)/mod.da

    updiag          = 0
    for j = 1:mod.J
        updiag      = vcat(updiag, Z[1:mod.I-1,j], 0)      # Note: Z_I,j = 0 for all j
    end
    updiag = updiag[2:end-1,1]
    centdiag = reshape(Y,mod.I*mod.J,1)
    lowdiag=X[2:mod.I,1]
    for j=2:mod.J
        lowdiag=vcat(lowdiag, 0, X[2:mod.I,j])          # Note: X_1,j = 0 for all j
    end

    B              = spdiagm(centdiag[:,1],0,mod.I*mod.J,mod.I*mod.J) + spdiagm(updiag[:,1],1,mod.I*mod.J,mod.I*mod.J) + spdiagm(lowdiag[:,1],-1,mod.I*mod.J,mod.I*mod.J)
    A              = B + mod.Cswitch
    InvMat         = (1/mod.Delta + mod.rho)*speye(mod.I*mod.J) - A

    u_stacked      = reshape(u,mod.I*mod.J,1)
    V_stacked      = reshape(V,mod.I*mod.J,1)

    # Set up and solve linear system of eqns    
    b              = u_stacked + V_stacked/mod.Delta
    V_stacked      = InvMat\b  
    V              = reshape(V_stacked,mod.I,mod.J)
    Vchange        = V - v
    v              = V

    dist[n] = maximum(maximum(abs(Vchange)))
    if dist[n]<mod.crit
      @printf("-----------------------\n")
      @printf("Interest rate = %f\n", r)
      @printf("Value Function Converged, Iteration = %i\n", n)
      break
    end
  end


  # Solve Kolmogorov Forward equation
  g = KFE(A)
    
  # Pack up output
  g_r[:,:,ir]     = g
  adot[:,:,ir]    = mod.zz + r.*mod.aa - c
  V_r[:,:,ir]     = V
  S[ir]           = ((g'*mod.agrid*mod.da)'*ones(mod.I,1)*mod.dz)[1,1]

  # Check for asset market clearing, update the interest rate
  r_end = ir

  if S[ir]>mod.crit_S
    @printf("Excess Supply at r = %0.2f\n", r)
    mod.rmax = r
    r = 0.5*(r+mod.rmin)
  elseif S[ir]<-mod.crit_S
    @printf("Excess Demand at r = %0.2f\n", r)
    mod.rmin = r
    r = 0.5*(r+mod.rmax)
  elseif abs(S[ir])<mod.crit_S
    @printf("Equilibrium Found, Interest rate = %0.2f\n", r)
    break
  end

end


-----------------------
Interest rate = 0.030000
Value Function Converged, Iteration = 9
Excess Demand at r = 0.03
-----------------------
Interest rate = 0.037500
Value Function Converged, Iteration = 5
Excess Supply at r = 0.04
-----------------------
Interest rate = 0.033750
Value Function Converged, Iteration = 5
Excess Demand at r = 0.03
-----------------------
Interest rate = 0.035625
Value Function Converged, Iteration = 4
Excess Supply at r = 0.04
-----------------------
Interest rate = 0.034688
Value Function Converged, Iteration = 4
Excess Supply at r = 0.03
-----------------------
Interest rate = 0.034219
Value Function Converged, Iteration = 4
Excess Supply at r = 0.03
-----------------------
Interest rate = 0.033984
Value Function Converged, Iteration = 4
Excess Supply at r = 0.03
-----------------------
Interest rate = 0.033867
Value Function Converged, Iteration = 3
Excess Demand at r = 0.03
-----------------------
Interest rate = 0.033926
Value Function Converged, Itera

### Graphs

Finally, I use the Gadfly package to plot the savings policy function across income states, and to plot the distribution of assets across income states. One problem here is that Gadfly cannot produce 3D graphs, and so we cannot visualize the distributions across income states as easily as in other programs, such as Matlab. I opt for two alternatives here: I plot a selection of 3 income levels, and also plot a contour plot for the stationary distributions. 

In [7]:
using Gadfly

#### Savings policy function

Here, I plot a selection of 3 income states, with green being the highest, red the next highest, and blue the lowest. The savings policy functions differ across income states. In the low income state, agents are always dissaving, regardless of their current wealth/assets. 

In the high income states, agents are saving over most of the asset state space. These agents will dissave when assets are very high. 

We can see that high income agents have a precautionary saving motive; they are self-insuring against the possibility of hitting the borrowing constraint if/when they enter the low income state. Low income agents always dissave in order to smooth consumption across income states. 

In [11]:
draw(SVG(20cm, 15cm), 
plot(layer(x=mod.agrid,y=adot[:,70,r_end],Geom.line,Theme(line_width=1.5mm)), 
layer(x=mod.agrid,y=adot[:,80,r_end],Geom.line, Theme(line_width=1.5mm,default_color=colorant"red")),
layer(x=mod.agrid,y=adot[:,90,r_end],Geom.line, Theme(line_width=1.5mm,default_color=colorant"green")),
xintercept=[mod.amin], yintercept=[0], Geom.hline(color=colorant"black", size=0.5mm), Geom.vline(color=colorant"black", size=0.5mm),
Guide.title("Savings policy function"),
Guide.xlabel("Wealth, <i>a"),
Guide.ylabel("Savings, <i>s<sub>i</sub>(a)"),
Coord.cartesian(xmin=mod.amin, xmax=mod.amax), 
Theme(major_label_font_size=6mm,
minor_label_font_size=4mm)) )

#### Stationary distributions

In the figure below we see the stationary distribution of agents across the three selected income states. Note that in the low income state (blue), there is a Dirac mass point on the borrowing constraint. That is, a large proportion of the low income agents are at the borrowing constraint in the stationary distribution. 

We can see that the higher income state distributions are shifted right relative to the low income state distribution. This reflects the fact that high income state agents are saving while in that state.

In [13]:
draw(SVG(20cm, 15cm), 
plot(layer(x=mod.agrid,y=g_r[:,70,r_end],Geom.line,Theme(line_width=1.5mm)), 
layer(x=mod.agrid,y=g_r[:,80,r_end],Geom.line, Theme(line_width=1.5mm,default_color=colorant"red")),
layer(x=mod.agrid,y=g_r[:,90,r_end],Geom.line, Theme(line_width=1.5mm,default_color=colorant"green")),
xintercept=[mod.amin], Geom.vline(color=colorant"black", size=0.5mm), 
yintercept=[0], Geom.hline(color=colorant"black", size=0.5mm),
Guide.title("Stationary distributions"),
Guide.xlabel("Wealth, <i>a"),
Guide.ylabel("Density, <i>g<sub>i</sub>(a)"),
Coord.cartesian(xmin=mod.amin, xmax=1), 
Theme(major_label_font_size=6mm,
minor_label_font_size=4mm)) )

#### Stationary distributions: contour plot

In the final figure, I plot a contour across all income and asset states. Unfortunately, the figure is hard to interpret (a 3D would have been much easier to do so). 


In [14]:
draw(SVG(20cm, 15cm), 
plot(z=adot[:,:,r_end], x = mod.agrid, y = mod.zgrid, Geom.contour, Theme(line_width=1.5mm),Guide.title("Stationary distribution"),
Guide.xlabel("Wealth, <i>a"),
Guide.ylabel("Income, <i>z"),
Coord.cartesian(xmin=mod.amin, xmax=2), Theme(major_label_font_size=6mm,
minor_label_font_size=4mm)) )